In [2]:
import pandas as pd

df = pd.read_parquet("transactions_clean.parquet")

# Datetime
df["date"] = pd.to_datetime(df["date"])

# ID columns
df["client_id"]   = df["client_id"].astype("int32")
df["card_id"]     = df["card_id"].astype("int32")
df["merchant_id"] = df["merchant_id"].astype("int32")
df["mcc"]         = df["mcc"].astype("int16")

# Amount
df["amount"] = df["amount"].astype("float32")

# Categorical features
for c in ["use_chip", "merchant_city", "merchant_state", "zip"]:
    df[c] = df[c].astype("category")

# Error flags
for c in [
    "has_error",
    "err_card_credential",
    "err_authentication",
    "err_financial",
    "err_system"
]:
    df[c] = df[c].astype("int8")

# Target
df["fraud"] = df["fraud"].astype("int8")

df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8851561 entries, 0 to 8851560
Data columns (total 16 columns):
 #   Column               Dtype         
---  ------               -----         
 0   date                 datetime64[ns]
 1   client_id            int32         
 2   card_id              int32         
 3   amount               float32       
 4   use_chip             category      
 5   merchant_id          int32         
 6   merchant_city        category      
 7   merchant_state       category      
 8   zip                  category      
 9   mcc                  int16         
 10  fraud                int8          
 11  has_error            int8          
 12  err_card_credential  int8          
 13  err_authentication   int8          
 14  err_financial        int8          
 15  err_system           int8          
dtypes: category(4), datetime64[ns](1), float32(1), int16(1), int32(3), int8(6)
memory usage: 323.8 MB


In [5]:
trans = df.copy()
user = pd.read_csv("user_common.csv")
card = pd.read_csv("card_common.csv")

In [9]:
import os
import numpy as np

# =========================
# 0) 설정
# =========================
base_path = "data/sampling"
ratios = [1, 3, 5, 7]
seed = 42

os.makedirs(base_path, exist_ok=True)

# =========================
# 1) fraud / nonfraud split
# =========================
fraud_df = trans[trans["fraud"] == 1].copy()
nonfraud_df = trans[trans["fraud"] == 0].copy()

n_fraud = len(fraud_df)
print("n_fraud:", n_fraud, "n_nonfraud:", len(nonfraud_df))

# nonfraud 셔플 (겹치지 않게 자를 거라 중요)
nonfraud_df = nonfraud_df.sample(frac=1, random_state=seed).reset_index(drop=True)

# =========================
# 2) ratio별 nonfraud chunk 생성 + 저장
# =========================
cursor = 0
meta = []

for r in ratios:
    need = r * n_fraud

    if cursor + need > len(nonfraud_df):
        print(f"[STOP] nonfraud 부족: 1:{r} 필요 {need}, 남은 {len(nonfraud_df)-cursor}")
        break

    nonfraud_part = nonfraud_df.iloc[cursor:cursor+need].copy()
    cursor += need

    df_r = (
        pd.concat([fraud_df, nonfraud_part], axis=0)
        .sample(frac=1, random_state=seed)
        .reset_index(drop=True)
    )

    folder = f"{base_path}/fraud_1_to_{r}"
    os.makedirs(folder, exist_ok=True)

    out_path = f"{folder}/data.parquet"
    df_r.to_parquet(out_path, index=False)

    meta.append({
        "dataset": f"fraud_1_to_{r}",
        "fraud": int((df_r["fraud"] == 1).sum()),
        "nonfraud": int((df_r["fraud"] == 0).sum()),
        "total": int(len(df_r)),
        "path": out_path
    })

# =========================
# 3) 남은 nonfraud 저장 (선택)
# =========================
remainder = nonfraud_df.iloc[cursor:].copy()
remainder_path = f"{base_path}/remainder_nonfraud.parquet"
remainder.to_parquet(remainder_path, index=False)

meta_df = pd.DataFrame(meta)
print(meta_df)
print("remainder_nonfraud:", len(remainder), "saved ->", remainder_path)


n_fraud: 10013 n_nonfraud: 8841548
        dataset  fraud  nonfraud  total  \
0  fraud_1_to_1  10013     10013  20026   
1  fraud_1_to_3  10013     30039  40052   
2  fraud_1_to_5  10013     50065  60078   
3  fraud_1_to_7  10013     70091  80104   

                                      path  
0  data/sampling/fraud_1_to_1/data.parquet  
1  data/sampling/fraud_1_to_3/data.parquet  
2  data/sampling/fraud_1_to_5/data.parquet  
3  data/sampling/fraud_1_to_7/data.parquet  
remainder_nonfraud: 8681340 saved -> data/sampling/remainder_nonfraud.parquet


In [10]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

base_path = "data/sampling"
ratio_names = ["fraud_1_to_1", "fraud_1_to_3", "fraud_1_to_5", "fraud_1_to_7"]

def summarize_results(res_df: pd.DataFrame):
    summary = (
        res_df
        .groupby("variable")
        .agg(
            sign_consistent=("beta", lambda x: len(set(np.sign(x))) == 1),
            significant_cnt=("pvalue", lambda x: (x < 0.05).sum()),
            mean_or=("odds_ratio", "mean"),
            min_or=("odds_ratio", "min"),
            max_or=("odds_ratio", "max"),
        )
        .reset_index()
        .sort_values(["sign_consistent", "significant_cnt"], ascending=False)
    )
    return summary


## 4.1 WHEN — 시간 기반 분석

### 목적

- 정상 거래 대비 **시간적 패턴 이탈 여부** 식별

### 사용 컬럼

**Transactions**

- `date`

**Card**

- `acct_open_date`
- `expires`
- `year_pin_last_changed`

**User**

- `birth_year` (나이 계산용)

### 제외/축소 컬럼

- `birth_month`
- `retirement_age`

### 주요 가설

- 비정상 시간대(야간, 새벽) 거래에서 Fraud 발생률 증가
- 카드 개설 직후 또는 만료 임박 시점의 거래 위험 증가
- PIN 장기간 미변경 카드에서 Fraud 발생률 증가

### 파생 피처 후보

- 거래 시각 관련: `hour`, `is_night`
- 카드 상태: `days_since_card_open`, `days_to_expire`
- 보안 상태: `years_since_pin_change`
- 사용자 상태: `age_at_transaction`


### **가설1: OFF-HOUR에서 Fraud 증가? (타깃=fraud)**

In [11]:
results = []

for name in ratio_names:
    df = pd.read_parquet(f"{base_path}/{name}/data.parquet")

    df["hour"] = df["date"].dt.hour
    df["is_off_hour"] = ((df["hour"] <= 5) | (df["hour"] >= 22)).astype(int)

    y = df["fraud"].astype(int)

    X = df[["is_off_hour", "amount", "use_chip"]].copy()
    X = pd.get_dummies(X, drop_first=True)
    X = X.replace([np.inf, -np.inf], np.nan).fillna(0)
    X = sm.add_constant(X, has_constant="add").astype(float)

    model = sm.Logit(y, X).fit(disp=False)

    var = "is_off_hour"
    results.append({
        "dataset": name,
        "variable": var,
        "beta": float(model.params[var]),
        "odds_ratio": float(np.exp(model.params[var])),
        "pvalue": float(model.pvalues[var]),
    })

res_df = pd.DataFrame(results)
print(res_df)
print(summarize_results(res_df))


        dataset     variable      beta  odds_ratio        pvalue
0  fraud_1_to_1  is_off_hour -0.790488    0.453624  1.161750e-22
1  fraud_1_to_3  is_off_hour -0.765320    0.465185  1.169183e-34
2  fraud_1_to_5  is_off_hour -0.841352    0.431127  2.443010e-51
3  fraud_1_to_7  is_off_hour -0.891096    0.410206  8.605636e-64
      variable  sign_consistent  significant_cnt   mean_or    min_or    max_or
0  is_off_hour             True                4  0.440035  0.410206  0.465185


off-hour 거래는 정상 시간대 거래에 비해
Fraud일 확률이 약 55~60% 낮다

(OR ≈ 0.44 → 1 − 0.44 ≈ 56%)

*비정상 시간대(야간·새벽) 거래에서 Fraud 발생률 증가 -> 가설 기각*

> **Fraud는 비정상 시간대의 이벤트라기보다, 정상 소비 패턴 안에서 발생하는 구조적 현상이다.**

### **가설2: 카드 개설 직후/만료 임박 거래가 더 위험? (타깃=fraud)**

In [12]:
# card 전처리 
card["acct_open_date"] = pd.to_datetime(card["acct_open_date"])
card["expires"] = pd.to_datetime(card["expires"])

results = []

for name in ratio_names:
    df = pd.read_parquet(f"{base_path}/{name}/data.parquet")

    df["hour"] = df["date"].dt.hour
    df["is_off_hour"] = ((df["hour"] <= 5) | (df["hour"] >= 22)).astype(int)

    df = df.merge(
        card[["id", "acct_open_date", "expires"]],
        left_on="card_id",
        right_on="id",
        how="left"
    )

    df["days_since_open"] = (df["date"] - df["acct_open_date"]).dt.days
    df["days_to_expire"] = (df["expires"] - df["date"]).dt.days

    df = df.dropna(subset=["days_since_open", "days_to_expire"])
    df = df[(df["days_since_open"] >= 0) & (df["days_to_expire"] >= -365)]

    y = df["fraud"].astype(int)

    X = df[["days_since_open", "days_to_expire", "amount", "use_chip", "is_off_hour"]].copy()
    X = pd.get_dummies(X, drop_first=True)
    X = X.replace([np.inf, -np.inf], np.nan).fillna(0)
    X = sm.add_constant(X, has_constant="add").astype(float)

    model = sm.Logit(y, X).fit(disp=False)

    for var in ["days_since_open", "days_to_expire"]:
        results.append({
            "dataset": name,
            "variable": var,
            "beta": float(model.params[var]),
            "odds_ratio": float(np.exp(model.params[var])),
            "pvalue": float(model.pvalues[var]),
        })

res_df = pd.DataFrame(results)
print(res_df)
print(summarize_results(res_df))


/tmp/ipykernel_3347538/1887910223.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  card["acct_open_date"] = pd.to_datetime(card["acct_open_date"])
/tmp/ipykernel_3347538/1887910223.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  card["expires"] = pd.to_datetime(card["expires"])


        dataset         variable      beta  odds_ratio        pvalue
0  fraud_1_to_1  days_since_open -0.000067    0.999933  6.168393e-06
1  fraud_1_to_1   days_to_expire  0.000121    1.000121  3.991284e-09
2  fraud_1_to_3  days_since_open -0.000077    0.999923  6.942762e-12
3  fraud_1_to_3   days_to_expire  0.000181    1.000181  1.307703e-31
4  fraud_1_to_5  days_since_open -0.000075    0.999925  4.098268e-14
5  fraud_1_to_5   days_to_expire  0.000198    1.000198  1.026477e-48
6  fraud_1_to_7  days_since_open -0.000077    0.999923  3.698385e-17
7  fraud_1_to_7   days_to_expire  0.000194    1.000194  2.085107e-53
          variable  sign_consistent  significant_cnt   mean_or    min_or  \
0  days_since_open             True                4  0.999926  0.999923   
1   days_to_expire             True                4  1.000173  1.000121   

     max_or  
0  0.999933  
1  1.000198  


> **카드 생애 변수(개설 이후 기간, 만료까지 남은 기간)는**\
> **Fraud 발생과 통계적으로는 연관되지만,**\
> **실질적인 위험 설명력은 매우 제한적이다.**

### **가설3: PIN 변경 시점(구간)과 Fraud 위험 (타깃=fraud)**

In [13]:
card["year_pin_last_changed"] = pd.to_numeric(card["year_pin_last_changed"], errors="coerce")

results = []

for name in ratio_names:
    df = pd.read_parquet(f"{base_path}/{name}/data.parquet")

    # off-hour 재생성
    df["hour"] = df["date"].dt.hour
    df["is_off_hour"] = ((df["hour"] <= 5) | (df["hour"] >= 22)).astype(int)

    # card merge
    df = df.merge(
        card[["id", "year_pin_last_changed"]],
        left_on="card_id",
        right_on="id",
        how="left"
    )

    df["txn_year"] = df["date"].dt.year
    df["years_since_pin_change"] = df["txn_year"] - df["year_pin_last_changed"]

    df = df.dropna(subset=["years_since_pin_change"])
    df = df[df["years_since_pin_change"] >= 0]

    df["pin_age_grp"] = pd.cut(
        df["years_since_pin_change"],
        bins=[0, 1, 3, 5, 10, 50],
        labels=["≤1y", "1–3y", "3–5y", "5–10y", "10y+"]
    )

    y = df["fraud"].astype(int)

    X = df[["pin_age_grp", "amount", "use_chip", "is_off_hour"]].copy()
    X = pd.get_dummies(X, drop_first=True)
    X = X.replace([np.inf, -np.inf], np.nan).fillna(0)
    X = sm.add_constant(X, has_constant="add").astype(float)

    model = sm.Logit(y, X).fit(disp=False)

    for col in X.columns:
        if col.startswith("pin_age_grp_"):
            results.append({
                "dataset": name,
                "variable": col,
                "beta": float(model.params[col]),
                "odds_ratio": float(np.exp(model.params[col])),
                "pvalue": float(model.pvalues[col]),
            })

res_df = pd.DataFrame(results)
print(res_df.sort_values(["variable", "dataset"]).reset_index(drop=True))
print(summarize_results(res_df))


         dataset           variable      beta  odds_ratio        pvalue
0   fraud_1_to_1   pin_age_grp_10y+ -0.922387    0.397569  1.572301e-05
1   fraud_1_to_3   pin_age_grp_10y+ -1.243273    0.288439  2.789220e-14
2   fraud_1_to_5   pin_age_grp_10y+ -1.308679    0.270177  1.160098e-18
3   fraud_1_to_7   pin_age_grp_10y+ -1.147027    0.317580  1.230001e-15
4   fraud_1_to_1   pin_age_grp_1–3y  0.072251    1.074925  2.776708e-01
5   fraud_1_to_3   pin_age_grp_1–3y  0.078571    1.081740  1.246203e-01
6   fraud_1_to_5   pin_age_grp_1–3y  0.027230    1.027604  5.489059e-01
7   fraud_1_to_7   pin_age_grp_1–3y  0.000004    1.000004  9.999158e-01
8   fraud_1_to_1   pin_age_grp_3–5y  0.276146    1.318040  6.746802e-05
9   fraud_1_to_3   pin_age_grp_3–5y  0.129340    1.138077  1.449202e-02
10  fraud_1_to_5   pin_age_grp_3–5y  0.130340    1.139215  5.510451e-03
11  fraud_1_to_7   pin_age_grp_3–5y  0.086732    1.090605  4.744239e-02
12  fraud_1_to_1  pin_age_grp_5–10y -0.416365    0.659440  4.017

> **PIN 변경 이후 3–5년 구간에서 Fraud 발생 확률이 가장 높으며,**\
**PIN 변경 후 10년 이상 경과한 카드는 Fraud 발생 확률이 유의하게 낮다.**


## 4.2 WHERE — 위치 기반 분석

### 목적

- 거래 위치의 **공간적 일관성 여부** 평가

### 사용 컬럼

**Transactions**

- `merchant_state`
- `zip`
- `merchant_city` (보조)

**User**

- `latitude`
- `longitude`

### 제외/축소 컬럼

- `address` (직접 사용 X)
- `merchant_city`는 대표 위치 정보로 축소 가능

### 주요 가설

- 거주지와 지리적으로 먼 거래일수록 Fraud 위험 증가
- 단시간 내 지역 급변 거래 패턴에서 Fraud 증가
- `ONLINE` 거래는 오프라인 거래와 상이한 위험 분포를 가짐

### 파생 피처 후보

- `geo_distance(user ↔ merchant)`
- `is_out_of_state`
- `is_online`
- `location_change_rate`

In [14]:
trans["merchant_state"].nunique()

53

### **가설4: 주 활동 지역이 아닐 경우 fraud일 확률이 높다**

In [16]:
eda_rows = []

for name in ratio_names:
    df_s = pd.read_parquet(f"{base_path}/{name}/data.parquet")

    # is_online
    df_s["is_online"] = (
        (df_s["merchant_state"] == "ONLINE") |
        (df_s["zip"] == "ONLINE")
    ).astype(int)

    # home_state merge
    df_s = df_s.merge(
        home_state[["client_id", "home_state"]],
        on="client_id",
        how="left"
    )

    # is_out_of_state
    df_s["is_out_of_state"] = (
        (df_s["merchant_state"] != df_s["home_state"]) &
        (df_s["is_online"] == 0)
    ).astype(int)

    # EDA: fraud rate by out_of_state
    grp = (
        df_s.groupby("is_out_of_state")["fraud"]
        .mean()
        .reset_index()
    )

    grp["dataset"] = name
    eda_rows.append(grp)

eda_df = pd.concat(eda_rows, ignore_index=True)
eda_df


,is_out_of_state,fraud,dataset
0,0,0.514629,fraud_1_to_1
1,1,0.269070,fraud_1_to_1
2,0,0.260384,fraud_1_to_3
3,1,0.113428,fraud_1_to_3
4,0,0.174248,fraud_1_to_5
5,1,0.072038,fraud_1_to_5
6,0,0.130998,fraud_1_to_7
7,1,0.052468,fraud_1_to_7


> **사용자 주 활동 지역을 벗어났다는 사실 자체는 Fraud 위험을 설명하지 못한다.**

### **가설5: ONLINE` 거래는 오프라인 거래와 상이한 위험 분포를 가짐**

In [17]:
eda_online = []

for name in ratio_names:
    df_s = pd.read_parquet(f"{base_path}/{name}/data.parquet")

    df_s["is_online"] = (
        (df_s["merchant_state"] == "ONLINE") |
        (df_s["zip"] == "ONLINE")
    ).astype(int)

    grp = (
        df_s.groupby("is_online")["fraud"]
        .mean()
        .reset_index()
    )

    grp["dataset"] = name
    eda_online.append(grp)

eda_online_df = pd.concat(eda_online, ignore_index=True)
eda_online_df


,is_online,fraud,dataset
0,0,0.122615,fraud_1_to_1
1,1,0.881249,fraud_1_to_1
2,0,0.044347,fraud_1_to_3
3,1,0.718060,fraud_1_to_3
4,0,0.027198,fraud_1_to_5
5,1,0.596927,fraud_1_to_5
6,0,0.019536,fraud_1_to_7
7,1,0.518271,fraud_1_to_7


> **Fraud는 물리적 위치의 이탈보다는,** \
> **공간 정보가 사라진 거래 환경(ONLINE)에서 구조적으로 발생한다.**

In [ ]:
results = []

for name in ratio_names:
    df = pd.read_parquet(f"{base_path}/{name}/data.parquet")

    # features
    df["is_online"] = (
        (df["merchant_state"] == "ONLINE") |
        (df["zip"] == "ONLINE")
    ).astype(int)

    df["hour"] = df["date"].dt.hour
    df["is_off_hour"] = ((df["hour"] <= 5) | (df["hour"] >= 22)).astype(int)

    y = df["fraud"].astype(int)

    X = df[["is_online", "amount", "use_chip", "is_off_hour"]].copy()
    X = pd.get_dummies(X, drop_first=True)
    X = X.replace([np.inf, -np.inf], np.nan).fillna(0)
    X = sm.add_constant(X, has_constant="add").astype(float)

    # regularized logit (L2)
    model = sm.Logit(y, X).fit_regularized(
        method="l1",      # l1로 두되
        alpha=1e-6,       # alpha를 아주 작게(거의 MLE처럼)
        disp=False
    )

    # 결과 저장
    beta = float(model.params["is_online"])
    results.append({
        "dataset": name,
        "variable": "is_online",
        "beta": beta,
        "odds_ratio": float(np.exp(beta))
    })

res_df = pd.DataFrame(results)
res_df


/home/nakyung/.local/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/nakyung/.local/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))
/home/nakyung/.local/lib/python3.10/site-packages/statsmodels/base/l1_solvers_common.py:71: ConvergenceWarning: QC check did not pass for 6 out of 6 parameters
Try increasing solver accuracy or number of iterations, decreasing alpha, or switch solvers
  warnings.warn(message, ConvergenceWarning)
/home/nakyung/.local/lib/python3.10/site-packages/statsmodels/base/l1_solvers_common.py:144: ConvergenceWarning: Could not trim params automatically due to failed QC check. Trimming using trim_mode == 'size' will still work.
  warnings.warn(msg, ConvergenceWarning)
/home/nakyung/.local/lib/python3.10/site-packages/statsmodels/discrete/discrete_

,dataset,variable,beta,odds_ratio
0,fraud_1_to_1,is_online,2.084801,8.042994
1,fraud_1_to_3,is_online,2.091004,8.093038
2,fraud_1_to_5,is_online,2.072545,7.945015
3,fraud_1_to_7,is_online,2.095096,8.126223


## 4.3 WHAT — 금액·소비 합리성 분석

### 목적

- 거래 금액 및 업종이 **사용자 경제 수준 대비 합리적인지** 평가

### 사용 컬럼

**Transactions**

- `amount`
- `mcc`

**User**

- `yearly_income`
- `total_debt`
- `credit_score`

**Card**

- `credit_limit`
- `card_type`
- `card_brand`

### 제외/축소 컬럼

- `per_capita_income` (yearly_income으로 대체 가능)
- card_brand는 보조 변수로 사용

### 주요 가설

- 소득·한도 대비 과도한 거래 금액에서 Fraud 증가
- 사용자 과거 소비 이력에 없는 MCC에서 거래 발생 시 위험 증가
- 저신용 + 고액 거래 조합에서 Fraud 증가

### 파생 피처 후보

- `amount_to_credit_limit_ratio`
- `amount_to_income_ratio`
- `is_new_mcc_for_user`
- `mcc_risk_score`

## 4.4 HOW — 결제 방식 및 오류 패턴 분석

### 목적

- 결제 수단·오류 로그의 **물리적/논리적 일관성 검증**

### 사용 컬럼

**Transactions**

- `use_chip`
- `has_error`
- `err_card_credential`
- `err_authentication`
- `err_financial`
- `err_system`

**Card**

- `has_chip`

### 주요 가설

- 칩 미보유 카드에서 Chip Transaction 발생 시 이상 거래 가능성 증가
- 오류 발생 후 연속 재시도 거래에서 Fraud 위험 증가
- 인증 오류 후 즉시 성공 거래 패턴은 Fraud 가능성 증가

### 파생 피처 후보

- `chip_mismatch_flag`
- `error_count_recent_window`
- `error_then_success_flag`
- `error_type_diversity`




## 4.5 WHO — 사용자 특성 기반 분석

### 목적

- Fraud에 **상대적으로 취약한 사용자 특성 탐색**

### 사용 컬럼

**User**

- `gender`
- `num_credit_cards`
- `credit_score`
- `birth_year` (나이 계산용)

**Card**

- `num_cards_issued`
- `card_on_dark_web`

### 제외/축소 컬럼

- `birth_month`
- `retirement_age`

### 해석 주의사항

- 성별 등 인구통계 변수는 **단독 판단 근거로 사용하지 않음**
- 다른 패턴 변수와 결합하여 보조적으로 해석

### 주요 가설

- 다수 카드 보유 고객에서 관리 복잡도 증가로 Fraud 발생 가능성 증가
- 다크웹 노출 카드 보유 시 Fraud 위험 증가
- 저신용 사용자에서 Fraud 피해 취약성 증가
- 사용자 나이가 높을수록 fraud 취약성 증가

### 파생 피처 후보

- `cards_per_user`
- `darkweb_card_ratio`
- `user_risk_segment`
- `age_at_transaction`

---
# Result
---